In [1]:
import pandas as pd
import numpy as np

In [2]:
file_name = 'one_hot_pNaNs_agep_small.csv'
sort_by = 'AGEP'
ascending = False

In [3]:
# loading the delaware data 
df_h = pd.read_csv('/workspace/data/psam_h10_small.csv')
df_p = pd.read_csv('/workspace/data/psam_p10_small.csv')

In [4]:
h_colums_to_keep = ['SERIALNO', 'TEN', 'HHL', 'VEH', 'HINCP', 'R65', 'R18']
p_colums_to_keep = ['SERIALNO', 'SEX', 'AGEP', 'SCHL'] 


df_h_filtered = df_h.filter(items=h_colums_to_keep)
df_p_filtered = df_p.filter(items=p_colums_to_keep)


In [5]:
#df_h_clean = pd.concat([df_h_a_filtered, df_h_b_filtered, df_h_c_filtered, df_h_d_filtered])
print(df_h_filtered.head())
print(df_h_filtered.info())


        SERIALNO  TEN  HHL  VEH  HINCP  R65  R18
0  2022GQ0000148  NaN  NaN  NaN    NaN  NaN  NaN
1  2022GQ0000610  NaN  NaN  NaN    NaN  NaN  NaN
2  2022GQ0001044  NaN  NaN  NaN    NaN  NaN  NaN
3  2022GQ0001089  NaN  NaN  NaN    NaN  NaN  NaN
4  2022GQ0001666  NaN  NaN  NaN    NaN  NaN  NaN
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4963 entries, 0 to 4962
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   SERIALNO  4963 non-null   object 
 1   TEN       4094 non-null   float64
 2   HHL       4094 non-null   float64
 3   VEH       4094 non-null   float64
 4   HINCP     4094 non-null   float64
 5   R65       4094 non-null   float64
 6   R18       4094 non-null   float64
dtypes: float64(6), object(1)
memory usage: 271.5+ KB
None


In [6]:
# merge the two dataframes on the SERIALNO column
df = pd.merge(df_h_filtered, df_p_filtered, on='SERIALNO', how='inner')
print(df.head())
print(df.info())


        SERIALNO  TEN  HHL  VEH  HINCP  R65  R18  SEX  AGEP  SCHL
0  2022GQ0000148  NaN  NaN  NaN    NaN  NaN  NaN    2    84  21.0
1  2022GQ0000610  NaN  NaN  NaN    NaN  NaN  NaN    2    39  16.0
2  2022GQ0001044  NaN  NaN  NaN    NaN  NaN  NaN    2    19  19.0
3  2022GQ0001089  NaN  NaN  NaN    NaN  NaN  NaN    2    77  21.0
4  2022GQ0001666  NaN  NaN  NaN    NaN  NaN  NaN    2    18  16.0
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9641 entries, 0 to 9640
Data columns (total 10 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   SERIALNO  9641 non-null   object 
 1   TEN       9336 non-null   float64
 2   HHL       9336 non-null   float64
 3   VEH       9336 non-null   float64
 4   HINCP     9336 non-null   float64
 5   R65       9336 non-null   float64
 6   R18       9336 non-null   float64
 7   SEX       9641 non-null   int64  
 8   AGEP      9641 non-null   int64  
 9   SCHL      9417 non-null   float64
dtypes: float64(7), int64(2),

In [7]:
# create a ordering for the later appending of the households 
df_sorted = df.sort_values(by=['SERIALNO', sort_by], ascending=[True, ascending])
df['ORDER'] = df_sorted.groupby('SERIALNO').cumcount()+1
print(df.head())
print(df.info())


        SERIALNO  TEN  HHL  VEH  HINCP  R65  R18  SEX  AGEP  SCHL  ORDER
0  2022GQ0000148  NaN  NaN  NaN    NaN  NaN  NaN    2    84  21.0      1
1  2022GQ0000610  NaN  NaN  NaN    NaN  NaN  NaN    2    39  16.0      1
2  2022GQ0001044  NaN  NaN  NaN    NaN  NaN  NaN    2    19  19.0      1
3  2022GQ0001089  NaN  NaN  NaN    NaN  NaN  NaN    2    77  21.0      1
4  2022GQ0001666  NaN  NaN  NaN    NaN  NaN  NaN    2    18  16.0      1
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9641 entries, 0 to 9640
Data columns (total 11 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   SERIALNO  9641 non-null   object 
 1   TEN       9336 non-null   float64
 2   HHL       9336 non-null   float64
 3   VEH       9336 non-null   float64
 4   HINCP     9336 non-null   float64
 5   R65       9336 non-null   float64
 6   R18       9336 non-null   float64
 7   SEX       9641 non-null   int64  
 8   AGEP      9641 non-null   int64  
 9   SCHL      9417 non-nu

In [8]:
# Apply the grouping to each continuous variable

hincp_bins = [0, 5000, 10000, 15000, 20000, 25000, 35000, 50000, 75000, 100000, 150000, np.inf]
hincp_labels = ['under 5k', '5k-10k', '10k-15k', '15k-20k', '20k-25k', '25k-35k', '35k-50k', '50k-75k', '75k-100k', '100k-150k', '150k+']

# right = False: right open, left closed
df['HINCP'] = pd.cut(df['HINCP'], bins=hincp_bins, labels=hincp_labels, right=False, include_lowest=True)
#df = df[df['HINCP'] != 'excluded']

In [9]:
TEN_bins = [0, 2, 3]
TEN_labels = ['owned or mortgaged', 'rented']

df['TEN'] = pd.cut(df['TEN'], bins=TEN_bins, labels=TEN_labels)

In [10]:
R18_bins = [0, 1, 2]
R18_labels = ['no', 'yes']

# right = False: right open, left closed
df['R18'] = pd.cut(df['R18'], bins=R18_bins, labels=R18_labels, right=False, include_lowest=True)

R65_bins = [0, 1, 2]
R65_labels = ['no', 'yes']

df['R65'] = pd.cut(df['R65'], bins=R65_bins, labels=R65_labels, right=False, include_lowest=True)

# test dataframe to see correct binning

#test = pd.DataFrame()
#test['R18'] = [0, 1, 2]
#
#test['R18'] = pd.cut(test['R18'], bins=R18_bins, labels=R18_labels, right=False, include_lowest=True)
#
#print(test)

In [11]:
HHL_bins = [0, 1, 2, 3, 4, np.inf]
HHL_labels = ['english', 'spanish', 'other indo-european', 'asian and pacific island languages', 'other']

# test dataframe to see if bins are correct 
#test = pd.DataFrame()
#test['HHL'] = [0, 1, 2, 3]
#test['HHL'] = pd.cut(test['HHL'], bins=HHL_bins, labels=HHL_labels)

#print(test)

df['HHL'] = pd.cut(df['HHL'], bins=HHL_bins, labels=HHL_labels)

In [12]:
VEH_bins = [0, 1, 2, 3, 4, np.inf]
VEH_labels = ['no vehicles', '1 vehicle', '2 vehicles', '3 vehicles', '4 or more vehicles'] 

df['VEH'] = pd.cut(df['VEH'], bins=VEH_bins, labels=VEH_labels, include_lowest=True, right=False)


#test = pd.DataFrame()
#test['VEH'] = [0, 1, 2, 3, 4, 5]
#test['VEH'] = pd.cut(test['VEH'], bins=VEH_bins, labels=VEH_labels, include_lowest=True, right=False)
#
#print(test)

In [13]:
SEX_bins = [0, 1, 2]
SEX_labels = ['male', 'female']

df['SEX'] = pd.cut(df['SEX'], bins=SEX_bins, labels=SEX_labels)

In [14]:
AGEP_bins = [0, 5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60, 65, 70, 75, 80, 85, np.inf]
AGEP_labels = ['under 5', '5-9', '10-14', '15-19', '20-24', '25-29', '30-34', '35-39', '40-44', '45-49', '50-54', '55-59', '60-64', '65-69', '70-74', '75-79', '80-84', '85+']

df['AGEP'] = pd.cut(df['AGEP'], bins=AGEP_bins, labels=AGEP_labels, right=False, include_lowest=True)



In [15]:
SCHL_bins = [0, 4, 12, 18, 21, 22, np.inf]

SCHL_labels = ['other', 'less than high school', 'high school', 'college or associate', 'bachelor', 'graduate']
#
df['SCHL'] = pd.cut(df['SCHL'], bins=SCHL_bins, labels=SCHL_labels, right=False, include_lowest=True)


#test = pd.DataFrame()
#test['SCHL'] = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25]
#test['SCHL'] = pd.cut(test['SCHL'], bins=SCHL_bins, labels=SCHL_labels, right=False, include_lowest=True)
##
#print(test)

In [16]:
# Not executed for the dataset
df.dropna(inplace=True)

In [17]:
print(df.head())
print(df.info())

          SERIALNO                 TEN      HHL         VEH    HINCP  R65  \
305  2022HU0000526  owned or mortgaged  english   1 vehicle   5k-10k  yes   
306  2022HU0001117  owned or mortgaged  english  2 vehicles  50k-75k   no   
307  2022HU0001117  owned or mortgaged  english  2 vehicles  50k-75k   no   
308  2022HU0001117  owned or mortgaged  english  2 vehicles  50k-75k   no   
309  2022HU0001117  owned or mortgaged  english  2 vehicles  50k-75k   no   

     R18     SEX   AGEP         SCHL  ORDER  
305   no    male  70-74     bachelor      1  
306  yes  female  40-44  high school      2  
307  yes    male  45-49  high school      1  
308  yes  female  20-24  high school      3  
309  yes    male  15-19  high school      4  
<class 'pandas.core.frame.DataFrame'>
Index: 7033 entries, 305 to 9638
Data columns (total 11 columns):
 #   Column    Non-Null Count  Dtype   
---  ------    --------------  -----   
 0   SERIALNO  7033 non-null   object  
 1   TEN       7033 non-null   catego

In [18]:
# converting the df to having only one entry per serialno

static_cols = h_colums_to_keep
static_df = df[static_cols].drop_duplicates('SERIALNO')

#print(static_df.head())
print(static_df.info())

attributes = ['SEX', 'AGEP', 'SCHL']

pivoted_df = []

for attribute in attributes:
    tmp = df.pivot(index='SERIALNO', columns='ORDER', values=attribute)
    tmp.columns = [f'{attribute}_{col}' for col in tmp.columns]
    pivoted_df.append(tmp)

pivoted_df = pd.concat(pivoted_df, axis=1)
pivoted_df.reset_index(inplace=True)
#print(pivoted_df.head())
print(pivoted_df.info())

#df_flattened = pd.concat([static_df, pivoted_df], axis=0)#.drop('SERIALNO', axis=1)
df_flattened = pd.merge(static_df, pivoted_df, on='SERIALNO', how='inner').drop('SERIALNO', axis=1)
#print(df_flattened.head())
#print(df_flattened.info())




<class 'pandas.core.frame.DataFrame'>
Index: 3156 entries, 305 to 9636
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype   
---  ------    --------------  -----   
 0   SERIALNO  3156 non-null   object  
 1   TEN       3156 non-null   category
 2   HHL       3156 non-null   category
 3   VEH       3156 non-null   category
 4   HINCP     3156 non-null   category
 5   R65       3156 non-null   category
 6   R18       3156 non-null   category
dtypes: category(6), object(1)
memory usage: 69.0+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3156 entries, 0 to 3155
Data columns (total 43 columns):
 #   Column    Non-Null Count  Dtype   
---  ------    --------------  -----   
 0   SERIALNO  3156 non-null   object  
 1   SEX_1     3156 non-null   category
 2   SEX_2     2037 non-null   category
 3   SEX_3     1013 non-null   category
 4   SEX_4     537 non-null    category
 5   SEX_5     193 non-null    category
 6   SEX_6     61 non-null     category
 7   SEX_7 

In [19]:
print(df_flattened.head())
print(df_flattened.info())

                  TEN      HHL                 VEH      HINCP  R65  R18  \
0  owned or mortgaged  english           1 vehicle     5k-10k  yes   no   
1  owned or mortgaged  english          2 vehicles    50k-75k   no  yes   
2  owned or mortgaged  english          2 vehicles  100k-150k   no  yes   
3  owned or mortgaged  english  4 or more vehicles      150k+   no  yes   
4  owned or mortgaged  english          2 vehicles    50k-75k   no  yes   

    SEX_1   SEX_2   SEX_3   SEX_4  ...                 SCHL_5  \
0    male     NaN     NaN     NaN  ...                    NaN   
1    male  female  female    male  ...  less than high school   
2  female    male     NaN     NaN  ...                    NaN   
3    male  female    male    male  ...  less than high school   
4  female    male  female  female  ...                  other   

                  SCHL_6 SCHL_7 SCHL_8 SCHL_9 SCHL_10 SCHL_11 SCHL_12 SCHL_13  \
0                    NaN    NaN    NaN    NaN     NaN     NaN     NaN     NaN

In [20]:
# create one hot encoding

# only NaN category for the persons attributes
df_pivoted_cols = pivoted_df.columns.tolist()
df_pivoted_cols.remove('SERIALNO')
#print(df_pivoted_cols)
df_non_pivot = df_flattened.drop(columns=df_pivoted_cols)

df_non_pivot = pd.get_dummies(df_non_pivot, columns=df_non_pivot.columns, prefix_sep=':', dtype=int, dummy_na=False)

df_pivoted = df_flattened[df_pivoted_cols]

df_pivoted = pd.get_dummies(df_pivoted, columns=df_pivoted.columns, prefix_sep=':', dtype=int, dummy_na=True)

df_onehot = pd.concat([df_non_pivot, df_pivoted], axis=1) 



# use this command to get the non nan one hot encoding 
#df_onehot = pd.get_dummies(df_flattened, columns=df_flattened.columns, prefix_sep=':', dtype=int)


# create dummies for one hot encoding for all columns  -- Just override the previous df_onehot
#df_onehot = pd.get_dummies(df_flattened, columns=df_flattened.columns, prefix_sep=':', dtype=int, dummy_na=True)

print(list(df_onehot.columns))

['TEN:owned or mortgaged', 'TEN:rented', 'HHL:english', 'HHL:spanish', 'HHL:other indo-european', 'HHL:asian and pacific island languages', 'HHL:other', 'VEH:no vehicles', 'VEH:1 vehicle', 'VEH:2 vehicles', 'VEH:3 vehicles', 'VEH:4 or more vehicles', 'HINCP:under 5k', 'HINCP:5k-10k', 'HINCP:10k-15k', 'HINCP:15k-20k', 'HINCP:20k-25k', 'HINCP:25k-35k', 'HINCP:35k-50k', 'HINCP:50k-75k', 'HINCP:75k-100k', 'HINCP:100k-150k', 'HINCP:150k+', 'R65:no', 'R65:yes', 'R18:no', 'R18:yes', 'SEX_1:male', 'SEX_1:female', 'SEX_1:nan', 'SEX_2:male', 'SEX_2:female', 'SEX_2:nan', 'SEX_3:male', 'SEX_3:female', 'SEX_3:nan', 'SEX_4:male', 'SEX_4:female', 'SEX_4:nan', 'SEX_5:male', 'SEX_5:female', 'SEX_5:nan', 'SEX_6:male', 'SEX_6:female', 'SEX_6:nan', 'SEX_7:male', 'SEX_7:female', 'SEX_7:nan', 'SEX_8:male', 'SEX_8:female', 'SEX_8:nan', 'SEX_9:male', 'SEX_9:female', 'SEX_9:nan', 'SEX_10:male', 'SEX_10:female', 'SEX_10:nan', 'SEX_11:male', 'SEX_11:female', 'SEX_11:nan', 'SEX_12:male', 'SEX_12:female', 'SEX_12:

In [21]:
print(df_onehot.head())
print(df_onehot.info())

   TEN:owned or mortgaged  TEN:rented  HHL:english  HHL:spanish  \
0                       1           0            1            0   
1                       1           0            1            0   
2                       1           0            1            0   
3                       1           0            1            0   
4                       1           0            1            0   

   HHL:other indo-european  HHL:asian and pacific island languages  HHL:other  \
0                        0                                       0          0   
1                        0                                       0          0   
2                        0                                       0          0   
3                        0                                       0          0   
4                        0                                       0          0   

   VEH:no vehicles  VEH:1 vehicle  VEH:2 vehicles  ...  SCHL_13:bachelor  \
0                0              1 

In [22]:
# drop zero columns 
#df_onehot = df_onehot.loc[:, (df_onehot.sum() != 0)]
df_onehot.to_csv(f'/workspace/data/{file_name}', index=False)

In [23]:
# compute ration of 0s to 1s 

unique_values = np.unique(df_onehot.values, return_counts=True)
print(unique_values)

# compute the ratio of 0s to 1s
ratio = unique_values[1][1] / unique_values[1][0]
print(f'{ratio:.3f}')

(array([0, 1]), array([1215060,  151488]))
0.125
